# Cinemática Direta

In [1]:
import pandas as pd
import numpy as np

## Classes


### Elo

A classe elo representação o elo físico do rôbo. Cada elo é descrito por meio dos parâmetros da notação de Denavit-Hartenberg (DH)

* theta: Ângulo de rotação em torno do eixo z (graus)
* d: Distância ao longo do eixo z (cm)
* a: Comprimento do elo (cm)
* alpha: Ângulo de rotação em torno do eixo x comum (graus)
* phase: Fase do ângulo de rotação em torno do eixo z (graus)



In [2]:
class Elo:
  # Construtor da classe Elo
  def __init__(self,theta,d,a,alpha,phase):
    self.theta = theta
    self.d = d
    self.a = a
    self.alpha = alpha
    self.phase = phase

### Robo


O parâmetro variável da junta rotativa/torcional deve ser um string. Os Elos devem serem informado usando uma lista, como no exemplo abaixo:

```
  Elos = [['theta_1',d,a,alpha,phase],
          ['theta_2',d,a,alpha,phase]]
```

Exemplo:

```
  Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0]]
  
  robo = Robo('NomeRobo',Elos)


  

In [3]:
from numpy import array
from numpy import radians as npradians
from sympy import sin, cos, Matrix,symbols, simplify, nsimplify, eye,Symbol
from math import radians

class Robo:
  # Construtor da classe
  def __init__(self,nameBot,elos):
    self.nameBot = nameBot
    self.series = self.__series_link(elos)
    self.matrixForwardKinematics = self.__homogeneous_transformations()
    self.joinName = self.__join_names()
    self.__simplifForwardKinematics = None

  # Método privado para criar a cadeia cinemática de elos. Obs: tenho que deixar mais generalista. Apenas juntas rotativas estão valendo
  def __series_link(self,elos):
    series = []
    for elemento in elos:
      series.append(Elo(symbols(elemento[0], real = True),elemento[1],elemento[2],elemento[3],elemento[4]))
    return series

  # Método privado para criar uma matriz de transformação homogênea de DH genérica
  def __matrix_homogeneous_transformations(self,theta, d, a, alpha):
    alpha = radians(alpha)
    r = 5
    # Matriz TH para a notação denavit-hartenberg
    matrix = Matrix([
            [cos(theta), -sin(theta) * cos(alpha).round(r), sin(theta) * sin(alpha).round(r), a * cos(theta)],
            [sin(theta), cos(theta) * cos(alpha).round(r), -cos(theta) * sin(alpha).round(r), a * sin(theta)],
            [0, sin(alpha).round(r), cos(alpha).round(r), d],
            [0, 0, 0, 1]
        ])
    return matrix

  # Método privado que define a matriz de tranformação total do manipulador
  def __homogeneous_transformations(self):
    # Inicializa a matriz transformação homogênea
    matrix_TH = eye(4)
    # Percorre os elos
    for elo in self.series:
      matrix = self.__matrix_homogeneous_transformations(elo.theta+radians(elo.phase),elo.d,elo.a,elo.alpha)
      matrix_TH = matrix_TH @ matrix
    return matrix_TH

  # Método público que exibe e seta a matriz de transformação no formato reduzida
  def show_matrix_ForwardKinematics(self):
    # Simplifica uma unica vez e exibe nas proximas vezes a matriz homogenia
    if self.__simplifForwardKinematics is None:
      self.__simplifForwardKinematics = simplify(nsimplify(self.matrixForwardKinematics))
      self.matrixForwardKinematics = self.__simplifForwardKinematics
      return self.__simplifForwardKinematics
    else:
      return self.__simplifForwardKinematics

  # Método privado que coleta as variáveis das juntas
  def __join_names(self):
    # Declara a variável como um conjunto para que só exista apenas uma varivel
    joinName = []
    # Percorre a cadeia cinemática de elos
    for elo in self.series:
      # Analisa cada atributo da classe elo
      for att,val in elo.__dict__.items():
        # Amazena o nome da variavel do atributo que é uma variável simbólica
        if isinstance(val, Symbol):
          joinName.append(val)
    return joinName

  # Método que retorna a matriz de transform
  def frame_effector(self,pose):
    joinName = self.joinName
    pose = npradians(pose)
    #if len(joinName) == len(pose):
    data = dict(zip(joinName,pose))
    return self.matrixForwardKinematics.evalf(subs=data).tolist()
    #else:
      #print("Número de valores é diferente das variáveis")

## Mapeamento da Cinemática Direta

A base de dados é criada com base no mapeadomento da cinemática direta. Isso ocorre definindo a modelagem direta do manipulador. Ao passo que necessário criar o espaço de possibilidade de atuação das variáveis das juntas. Assim, cada conjunto de junta entra na modelagem direta que retorna a matriz de tranformação homogênea do sistema de coordenadas do efetuador, aqui nomeado espaço operacional. Através dessas desses dois espaços é possível mapear a cinemática direta do manipulador. 

In [4]:
# Define os parâmetros DH para cada elo
Elos = [['theta_1',10,0,90,0],
        ['theta_2',0,18,180,0],
        ['theta_3',0,18,-180,0],

        ['theta_4',0,0,90,90],
        ['theta_5',18,0,0,0]]

# Cria o robo por meio dos parâmetros
robo = Robo('Robotest',Elos)

In [5]:
robo.show_matrix_ForwardKinematics()

Matrix([
[sin(theta_1)*sin(theta_5) + cos(theta_1)*cos(theta_5)*cos(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000),  sin(theta_1)*cos(theta_5) - sin(theta_5)*cos(theta_1)*cos(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000), sin(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000)*cos(theta_1), 18*(sin(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000) + cos(theta_2) + cos(theta_2 - theta_3))*cos(theta_1)],
[sin(theta_1)*cos(theta_5)*cos(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000) - sin(theta_5)*cos(theta_1), -sin(theta_1)*sin(theta_5)*cos(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000) - cos(theta_1)*cos(theta_5), sin(theta_1)*sin(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000), 18*(sin(theta_2 - theta_3 + theta_4 + 15707963267949/10000000000000) + cos(theta_2) + cos(theta_2 - theta_3))*sin(theta_1)],
[                                         sin(theta_2 - theta_3 + theta_4 + 15707963267949/10000000

### Espaço das Juntas

In [6]:
# Limites físicos das juntas em graus
junta1 = [0, 120]
junta2 = [0, 120]
junta3 = [0, 120]
junta4 = [0, 120]
junta5 = [0, 0]

In [7]:
# passo de amostragem
step_1 = 5
step_2 = junta2[1]//10
step_3 = junta3[1]//10
step_4 = junta4[1]//10
step_5 = 1

In [8]:
# Criando uma lista para armazenar os dados
data_join = []

# Criando a matriz com todas as combinações de ângulos
for angulo_1 in range(junta1[0], junta1[1] + 1, step_1):
    for angulo_2 in range(junta2[0], junta2[1] + 1, step_2):
        for angulo_3 in range(junta3[0], junta3[1] + 1, step_3):
            for angulo_4 in range(junta4[0], junta4[1] + 1, step_4):
                data_join.append([angulo_1, angulo_2, angulo_3, angulo_4, junta5[0]])

In [9]:
df_join = pd.DataFrame(data_join, columns=robo.joinName)
df_join

,theta_1,theta_2,theta_3,theta_4,theta_5
0,0,0,0,0,0
1,0,0,0,12,0
2,0,0,0,24,0
3,0,0,0,36,0
4,0,0,0,48,0
...,...,...,...,...,...
33270,120,120,120,72,0
33271,120,120,120,84,0
33272,120,120,120,96,0
33273,120,120,120,108,0


### Espaço Operacional

In [10]:
# Cria uma matriz de 12 colunas vazia
data_operacional = np.empty((0, 12))

# Percorre a base de dados substituindo na matriz de transformação
for pose in data_join:
  # Cria um vetor da matriz de tranformação excluindo a ultima linha
  vetor = np.array(robo.frame_effector(pose)[:3]).reshape(-1)
  # Salva o vetor na matriz do espaço operacional
  data_operacional = np.vstack([data_operacional,vetor])

In [11]:
# Cria um dataframe da matriz do espaço operacional
df_pose = pd.DataFrame(data_operacional,columns=['R_11','R_12','R_13','p_x','R_21','R_22','R_23','p_y','R_31','R_32','R_33','p_z'])
df_pose

,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,-3.38076867830836e-15,0,1.00000000000000,54.0000000000000,0,-1.00000000000000,0,0,1.00000000000000,0,3.38076867830836e-15,10.0000000000001
1,-0.207911690817763,0,0.978147600733805,53.6066568132085,0,-1.00000000000000,0,0,0.978147600733805,0,0.207911690817763,13.7424104347197
2,-0.406736643075803,0,0.913545457642600,52.4438182375668,0,-1.00000000000000,0,0,0.913545457642600,0,0.406736643075803,17.3212595753645
3,-0.587785252292476,0,0.809016994374945,50.5623058987490,0,-1.00000000000000,0,0,0.809016994374945,0,0.587785252292476,20.5801345412646
4,-0.743144825477397,0,0.669130606358856,48.0443509144594,0,-1.00000000000000,0,0,0.669130606358856,0,0.743144825477397,23.3766068585931
...,...,...,...,...,...,...,...,...,...,...,...,...
33270,0.475528258147577,0.866025403784439,-0.154508497187472,-7.28115294937450,-0.823639103546333,0.500000000000000,0.267616567329815,12.6113268459966,0.309016994374944,0,0.951056516295155,42.7074745614327
33271,0.497260947684137,0.866025403784439,-0.0522642316338250,-5.44075616940885,-0.861281226008775,0.500000000000000,0.0905243046083336,9.42366611700996,0.104528463267650,0,0.994521895368274,43.4898513847488
33272,0.497260947684136,0.866025403784439,0.0522642316338284,-3.55924383059109,-0.861281226008774,0.500000000000000,-0.0905243046083395,6.16479115110984,-0.104528463267657,0,0.994521895368273,43.4898513847488
33273,0.475528258147576,0.866025403784439,0.154508497187475,-1.71884705062545,-0.823639103546331,0.500000000000000,-0.267616567329820,2.97713042212319,-0.309016994374951,0,0.951056516295153,42.7074745614326


In [12]:
# Concatena as matrix do espaço operacional com a do espaço da juntas
df_dataset = pd.concat([df_join, df_pose], axis=1)
df_dataset

,theta_1,theta_2,theta_3,theta_4,theta_5,R_11,R_12,R_13,p_x,R_21,R_22,R_23,p_y,R_31,R_32,R_33,p_z
0,0,0,0,0,0,-3.38076867830836e-15,0,1.00000000000000,54.0000000000000,0,-1.00000000000000,0,0,1.00000000000000,0,3.38076867830836e-15,10.0000000000001
1,0,0,0,12,0,-0.207911690817763,0,0.978147600733805,53.6066568132085,0,-1.00000000000000,0,0,0.978147600733805,0,0.207911690817763,13.7424104347197
2,0,0,0,24,0,-0.406736643075803,0,0.913545457642600,52.4438182375668,0,-1.00000000000000,0,0,0.913545457642600,0,0.406736643075803,17.3212595753645
3,0,0,0,36,0,-0.587785252292476,0,0.809016994374945,50.5623058987490,0,-1.00000000000000,0,0,0.809016994374945,0,0.587785252292476,20.5801345412646
4,0,0,0,48,0,-0.743144825477397,0,0.669130606358856,48.0443509144594,0,-1.00000000000000,0,0,0.669130606358856,0,0.743144825477397,23.3766068585931
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33270,120,120,120,72,0,0.475528258147577,0.866025403784439,-0.154508497187472,-7.28115294937450,-0.823639103546333,0.500000000000000,0.267616567329815,12.6113268459966,0.309016994374944,0,0.951056516295155,42.7074745614327
33271,120,120,120,84,0,0.497260947684137,0.866025403784439,-0.0522642316338250,-5.44075616940885,-0.861281226008775,0.500000000000000,0.0905243046083336,9.42366611700996,0.104528463267650,0,0.994521895368274,43.4898513847488
33272,120,120,120,96,0,0.497260947684136,0.866025403784439,0.0522642316338284,-3.55924383059109,-0.861281226008774,0.500000000000000,-0.0905243046083395,6.16479115110984,-0.104528463267657,0,0.994521895368273,43.4898513847488
33273,120,120,120,108,0,0.475528258147576,0.866025403784439,0.154508497187475,-1.71884705062545,-0.823639103546331,0.500000000000000,-0.267616567329820,2.97713042212319,-0.309016994374951,0,0.951056516295153,42.7074745614326


### Salvando

In [ ]:
path = '/content/drive/My Drive/TCC/Implementação/cinemática direta/dataset.csv'
df.to_csv(caminho_arquivo, index=False)